In [1]:
# Ensemble_Method_C.py
# Combined Script: Load CSV -> Feature Engineering -> Rolling Origin Stacking
# Uses feature set from Simple_Predictor_B
# Meta-Learner: Regularized Logistic Regression

import pandas as pd
import numpy as np
import time
import os
import warnings
import traceback
from datetime import datetime

# Feature Engineering Imports
# import pandas_ta as ta # No longer needed if using custom functions below

# Modeling Imports
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier, early_stopping, log_evaluation
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression # <-- Added Meta Learner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# --- Suppress Warnings ---
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore')

# --- Configuration ---

# Data Loading
CSV_FILE_PATH = r'C:\Users\mason\AVP\BTCUSDrec.csv'
SYMBOL_NAME = 'BTCUSD'

# Feature Selection (From Simple_Predictor_B)
SELECTED_FEATURE_NAMES = [
    'open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD',
    'price_range_pct', 'oc_change_pct', 'garman_klass_12h', 'parkinson_3h',
    'ma_3h', 'rolling_std_3h', 'lag_3h_price_return', 'lag_6h_price_return',
    'lag_12h_price_return', 'lag_24h_price_return', 'lag_48h_price_return',
    'lag_72h_price_return', 'lag_168h_price_return', 'volume_return_1h',
    'lag_3h_volume_return', 'lag_6h_volume_return', 'lag_12h_volume_return',
    'lag_24h_volume_return', 'ma_6h', 'ma_12h', 'ma_24h', 'ma_48h',
    'ma_72h', 'ma_168h', 'rolling_std_6h', 'rolling_std_12h',
    'rolling_std_24h', 'rolling_std_48h', 'rolling_std_72h',
    'rolling_std_168h', 'atr_14h', 'atr_24h', 'atr_48h', 'close_div_ma_24h',
    'close_div_ma_48h', 'close_div_ma_168h', 'ma12_div_ma48',
    'ma24_div_ma168', 'std12_div_std72', 'volume_btc_x_range',
    'rolling_std_3h_sq', 'price_return_1h_sq', 'rolling_std_12h_sqrt'
]
MODEL_FEATURE_COLS = [f for f in SELECTED_FEATURE_NAMES if f not in ['open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD']]

# Modeling & Walk-Forward
TARGET_THRESHOLD_PCT = 0.0
PREDICTION_WINDOW_HOURS = 12
PREDICTION_WINDOW_ROWS = PREDICTION_WINDOW_HOURS

# Walk-forward params (Adjusted slightly from B for variation)
TRAIN_WINDOW_HOURS = int(24 * 7 * 12) # ~12 weeks
TEST_WINDOW_HOURS = 24 * 21          # Evaluate over 3 weeks
STEP_HOURS = 24                      # Retrain daily

TRAIN_WINDOW_ROWS = TRAIN_WINDOW_HOURS
TEST_WINDOW_ROWS = TEST_WINDOW_HOURS
STEP_ROWS = STEP_HOURS

# Stacking Configuration
N_STACKING_FOLDS = 5

# --- Base Model Parameter Grids (For per-iteration tuning) ---
BASE_XGB_PARAM_GRID = {
    'max_depth': [2, 4],
    'n_estimators': [30, 70],
    'eta': [0.03, 0.05],
    'lambda': [1.5, 2.5]
}
BASE_LGBM_PARAM_GRID = {
    'max_depth': [3, 4],
    'n_estimators': [55, 95],
    'learning_rate': [0.04, 0.08],
    'subsample': [0.75, 0.9]
}
BASE_SVM_PARAM_GRID = {
    'C': [1.6, 3.2],
    'gamma': ['scale', 'auto'] # Keep gamma tuning
}

# --- Base Model Static/Fixed Hyperparameters ---
XGB_BASE_PARAMS = { # Defaults not covered by grid
    'objective': 'binary:logistic', 'eval_metric': 'logloss',
    'subsample': 0.8, 'colsample_bytree': 0.7, 'min_child_weight': 3,
    'gamma': 0.1, 'alpha': 0.1, 'random_state': 42, 'n_jobs': -1,
    'tree_method': 'hist', 'use_label_encoder': False,
}
LGBM_BASE_PARAMS = { # Defaults not covered by grid
    'objective': 'binary', 'metric': 'logloss', 'num_leaves': 8,
    'colsample_bytree': 0.7, 'min_child_samples': 5, 'reg_alpha': 0.1,
    'reg_lambda': 1.5, 'random_state': 42, 'n_jobs': -1,
    'boosting_type': 'gbdt', 'verbose': -1
}
SVM_BASE_PARAMS = { # Fixed SVM params
    'kernel': 'rbf', 'probability': True, 'max_iter': 5000,
    'random_state': 42, 'class_weight': 'balanced'
}

# --- Meta Learner Configuration (Logistic Regression) ---
META_LR_PARAM_GRID = {
    'C': [0.01, 0.1, 1, 10, 100] # Regularization strength (inverse)
}
META_LR_FIXED_PARAMS = {
    'penalty': 'l2', # Ridge regularization (can try 'l1' with 'liblinear')
    'solver': 'liblinear', # Good choice for L1/L2 and smaller datasets
    'class_weight': 'balanced', # Important for potentially imbalanced meta-features
    'max_iter': 1000, # Increase iterations for convergence
    'random_state': 123,
    'n_jobs': -1
}

# --- Probability Threshold Tuning Configuration ---
THRESHOLD_SEARCH_RANGE = np.arange(0.10, 0.90, 0.05)
META_VALIDATION_PCT = 0.25

# --- Feature Engineering Functions (Copied from request) ---
def garman_klass_volatility(open_, high, low, close, window):
    log_hl = np.log(high / low)
    log_co = np.log(close / open_)
    gk = 0.5 * (log_hl ** 2) - (2 * np.log(2) - 1) * (log_co ** 2)
    # Need to handle potential NaNs before rolling mean if inputs cause them
    gk = gk.fillna(0) # Or use rolling(..., min_periods=1) below? Let's fill first.
    rolling_mean = gk.rolling(window=window, min_periods=max(1, window // 2)).mean() # Allow fewer periods
    rolling_mean = rolling_mean.clip(lower=0) # Clip negative variances before sqrt
    return np.sqrt(rolling_mean)

def parkinson_volatility(high, low, window):
    log_hl_sq = np.log(high / low) ** 2
    # Handle potential NaNs
    log_hl_sq = log_hl_sq.fillna(0)
    rolling_sum = log_hl_sq.rolling(window=window, min_periods=max(1, window // 2)).sum()
    factor = 1 / (4 * np.log(2) * window)
    return np.sqrt(factor * rolling_sum)

def calculate_selected_features(df, symbol):
    """
    Calculates the features required by Simple_Predictor_B using custom methods.
    """
    print(f"Starting calculation for {len(SELECTED_FEATURE_NAMES)} target columns (incl. base)...")
    start_time = time.time()
    if df is None or len(df) < 3: return pd.DataFrame()
    df = df.copy()
    df['symbol'] = symbol

    # --- Timestamp and Index ---
    if 'timestamp' not in df.columns: print("Error: 'timestamp' column not found."); return pd.DataFrame()
    try: df['timestamp'] = pd.to_datetime(df['timestamp'])
    except Exception as e: print(f"Error converting timestamp: {e}"); return pd.DataFrame()
    df = df.sort_values('timestamp').dropna(subset=['timestamp'])
    df = df.set_index('timestamp', drop=False)

    # --- Volume Columns ---
    original_vol_btc_name = 'Volume BTC'; original_vol_usd_name = 'Volume USD'
    if original_vol_btc_name not in df.columns: df[original_vol_btc_name] = 0
    if original_vol_usd_name not in df.columns: df[original_vol_usd_name] = 0
    df[original_vol_btc_name] = pd.to_numeric(df[original_vol_btc_name], errors='coerce').fillna(0)
    df[original_vol_usd_name] = pd.to_numeric(df[original_vol_usd_name], errors='coerce').fillna(0)

    # --- Basic Checks (OHLC) ---
    required_ohlc = ['open', 'high', 'low', 'close']
    if not all(col in df.columns for col in required_ohlc): print(f"Error: Missing required OHLC columns: {required_ohlc}"); return pd.DataFrame()
    for col in required_ohlc: df[col] = pd.to_numeric(df[col], errors='coerce')
    if df[required_ohlc].isnull().any().any(): print("Warning: NaNs found in OHLC data. Dropping affected rows."); df = df.dropna(subset=required_ohlc)
    if df.empty: print("DataFrame empty after OHLC checks."); return pd.DataFrame()

    print("  Calculating features...")
    min_periods_rolling = 2 # Default min periods for rolling std/ma unless specified

    # --- Basic Calcs ---
    with np.errstate(divide='ignore', invalid='ignore'):
        df['price_range_pct'] = (df['high'] - df['low']) / df['close'].replace(0, np.nan) # Use close as denominator? Okay.
        df['oc_change_pct'] = (df['close'] - df['open']) / df['open'].replace(0, np.nan)
        df['price_return_1h_temp'] = df['close'].pct_change() # Temporary for std calc and sq feature
        df['volume_return_1h'] = df[original_vol_btc_name].pct_change()

    # --- Lagged Returns ---
    lag_price_hours = [3, 6, 12, 24, 48, 72, 168]
    lag_volume_hours = [3, 6, 12, 24]
    for hours in lag_price_hours: df[f'lag_{hours}h_price_return'] = df['close'].pct_change(periods=hours)
    for hours in lag_volume_hours: df[f'lag_{hours}h_volume_return'] = df[original_vol_btc_name].pct_change(periods=hours)

    # --- Moving Averages ---
    ma_hours = [3, 6, 12, 24, 48, 72, 168]
    for hours in ma_hours:
        df[f'ma_{hours}h'] = df['close'].rolling(window=hours, min_periods=max(min_periods_rolling, hours // 2)).mean()

    # --- Rolling Standard Deviations (Using price_return_1h_temp) ---
    std_hours = [3, 6, 12, 24, 48, 72, 168]
    if 'price_return_1h_temp' in df.columns:
        for hours in std_hours:
            # Multiply by 100 as per original code? Let's assume yes.
            df[f'rolling_std_{hours}h'] = df['price_return_1h_temp'].rolling(window=hours, min_periods=max(min_periods_rolling, hours // 2)).std() * 100
    else:
        for hours in std_hours: df[f'rolling_std_{hours}h'] = np.nan

    # --- ATR Calculation (Custom) ---
    print("    Calculating ATR, Garman-Klass, and Parkinson volatility features...")
    df['prev_close'] = df['close'].shift(1)
    df['high_minus_low'] = df['high'] - df['low']
    df['high_minus_prev_close'] = np.abs(df['high'] - df['prev_close'])
    df['low_minus_prev_close'] = np.abs(df['low'] - df['prev_close'])
    df['true_range'] = df[['high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close']].max(axis=1)
    for p in [14, 24, 48]:
         df[f'atr_{p}h'] = df['true_range'].rolling(window=p, min_periods=max(1, p // 2)).mean() # Allow fewer periods
    df = df.drop(columns=['prev_close', 'high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close', 'true_range'])

    # --- Garman-Klass and Parkinson Volatility (Custom) ---
    df['garman_klass_12h'] = garman_klass_volatility(df['open'], df['high'], df['low'], df['close'], window=12)
    df['parkinson_3h'] = parkinson_volatility(df['high'], df['low'], window=3)

    # --- Ratio Features ---
    with np.errstate(divide='ignore', invalid='ignore'):
        for hours in [24, 48, 168]:
            ma_col = f'ma_{hours}h'
            if ma_col in df.columns: df[f'close_div_ma_{hours}h'] = df['close'] / df[ma_col].replace(0, np.nan)
            else: df[f'close_div_ma_{hours}h'] = np.nan
        if 'ma_12h' in df.columns and 'ma_48h' in df.columns: df['ma12_div_ma48'] = df['ma_12h'] / df['ma_48h'].replace(0, np.nan)
        else: df['ma12_div_ma48'] = np.nan
        if 'ma_24h' in df.columns and 'ma_168h' in df.columns: df['ma24_div_ma168'] = df['ma_24h'] / df['ma_168h'].replace(0, np.nan)
        else: df['ma24_div_ma168'] = np.nan
        if 'rolling_std_12h' in df.columns and 'rolling_std_72h' in df.columns: df['std12_div_std72'] = df['rolling_std_12h'] / df['rolling_std_72h'].replace(0, np.nan)
        else: df['std12_div_std72'] = np.nan
        # Interaction
        if original_vol_btc_name in df.columns and 'price_range_pct' in df.columns: df['volume_btc_x_range'] = df[original_vol_btc_name] * df['price_range_pct']
        else: df['volume_btc_x_range'] = np.nan

    # --- Non-linear Transformations ---
    if 'rolling_std_3h' in df.columns: df['rolling_std_3h_sq'] = df['rolling_std_3h'] ** 2
    else: df['rolling_std_3h_sq'] = np.nan
    if 'price_return_1h_temp' in df.columns: df['price_return_1h_sq'] = (df['price_return_1h_temp'] ** 2) * 10000 # Scaling factor from original?
    else: df['price_return_1h_sq'] = np.nan
    if 'rolling_std_12h' in df.columns:
        epsilon = 1e-9 # Add epsilon before sqrt for stability
        df['rolling_std_12h_sqrt'] = np.sqrt(df['rolling_std_12h'].clip(lower=0) + epsilon)
    else: df['rolling_std_12h_sqrt'] = np.nan

    # --- Cleanup ---
    if 'price_return_1h_temp' in df.columns: df = df.drop(columns=['price_return_1h_temp'])

    print("  Assembling final dataframe...")
    final_cols_present = [col for col in SELECTED_FEATURE_NAMES if col in df.columns]
    df_final = df[final_cols_present + ['timestamp', 'symbol']].copy()
    missing_final_cols = set(SELECTED_FEATURE_NAMES) - set(df_final.columns)
    if missing_final_cols: print(f"  Final Warning: {len(missing_final_cols)} target columns missing: {missing_final_cols}")
    df_final = df_final.reset_index(drop=True)
    df_final = df_final.replace([np.inf, -np.inf], np.nan)
    end_time = time.time()
    actual_feature_count = len([col for col in df_final.columns if col not in ['timestamp', 'symbol', 'open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD']])
    print(f"Selected feature calculation finished. Returning {len(df_final)} rows, {len(df_final.columns)} columns "
          f"({actual_feature_count} calculated features). Took {end_time - start_time:.2f}s.")
    return df_final

# --- Helper: Grid Search for Base Models (Copied from request) ---
def grid_search_base_model(model_type, base_param_grid, X, y, scale_pos_weight_val):
    """Performs a simple 3-fold grid search using F1 score as metric."""
    best_score = -np.inf
    best_params = None
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    
    # Combine fixed and grid params for iteration
    fixed_params = {}
    if model_type == 'xgb': fixed_params = XGB_BASE_PARAMS
    elif model_type == 'lgbm': fixed_params = LGBM_BASE_PARAMS
    elif model_type == 'svm': fixed_params = SVM_BASE_PARAMS # SVM params handled slightly differently in pipeline

    full_param_list = []
    for grid_p in ParameterGrid(base_param_grid):
        p = fixed_params.copy()
        p.update(grid_p)
        full_param_list.append(p)
        
    if not full_param_list: # Handle case where grid is empty
        full_param_list.append(fixed_params)

    for params in full_param_list:
        scores = []
        # Check if y has at least 2 classes before splitting
        if len(np.unique(y)) < 2:
            print(f"    Skipping CV for {model_type}: Target has only one class.")
            return params, 0.0 # Return default params, score 0
            
        for train_idx, val_idx in cv.split(X, y):
            # Ensure validation split also has both classes
            y_val_inner = y.iloc[val_idx]
            if len(np.unique(y_val_inner)) < 2:
                scores.append(0) # Assign 0 if validation fold is single class
                continue
                
            X_train_inner, y_train_inner = X.iloc[train_idx], y.iloc[train_idx]
            X_val_inner = X.iloc[val_idx] # y_val_inner already defined

            try:
                if model_type == 'xgb':
                    # Remove SVM specific params if they sneaked in
                    params_xgb = {k: v for k, v in params.items() if k not in ['C', 'gamma', 'kernel', 'probability', 'max_iter', 'class_weight']}
                    model = XGBClassifier(**params_xgb, scale_pos_weight=scale_pos_weight_val)
                elif model_type == 'lgbm':
                     params_lgbm = {k: v for k, v in params.items() if k not in ['C', 'gamma', 'kernel', 'probability', 'max_iter', 'class_weight']}
                     model = LGBMClassifier(**params_lgbm, scale_pos_weight=scale_pos_weight_val)
                elif model_type == 'svm':
                     # Extract SVM specific params from the combined dict
                     svm_grid_params = {k: params[k] for k in base_param_grid if k in params}
                     model = Pipeline([
                        ('imputer', SimpleImputer(strategy='median')),
                        ('scaler', StandardScaler()),
                        ('svm', SVC(**fixed_params, **svm_grid_params)) # Pass fixed and grid params here
                     ])
                else: continue

                model.fit(X_train_inner, y_train_inner)
                y_pred_inner = model.predict(X_val_inner)
                score = f1_score(y_val_inner, y_pred_inner, zero_division=0)
                scores.append(score)
            except Exception as e:
                # print(f"      Error during {model_type} CV fit/predict with params {params}: {e}") # Optional debug
                scores.append(0) # Assign 0 score on error

        mean_score = np.mean(scores) if scores else 0
        if mean_score > best_score:
            best_score = mean_score
            # Store only the grid parameters that led to the best score
            best_params = {k: params[k] for k in base_param_grid if k in params}

    # Handle case where no parameters were found (e.g., all folds failed)
    if best_params is None and base_param_grid:
       best_params = list(ParameterGrid(base_param_grid))[0] # Return first grid combo as fallback

    return best_params, best_score

# --- Main Execution Block ---
if __name__ == "__main__":
    print("--- Ensemble_Method_C (Logistic Regression Meta-Learner) ---")
    print("--- 1. Data Loading & Initial Prep ---")
    try:
        print(f"Loading data from: {CSV_FILE_PATH}")
        col_names = ['unix', 'date', 'symbol_csv', 'open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD']
        df_raw = pd.read_csv(CSV_FILE_PATH, header=0, names=col_names)
        print(f"Raw data loaded. Shape: {df_raw.shape}")
        df_raw['timestamp'] = pd.to_datetime(df_raw['date'])
        df_raw = df_raw.drop(['unix', 'date', 'symbol_csv'], axis=1)
        df_raw = df_raw.sort_values('timestamp').reset_index(drop=True)
        if df_raw.empty: exit("DataFrame empty after loading. Exiting.")
        print(f"Initial data prep done. Shape: {df_raw.shape}")
    except Exception as e:
        print(f"Error loading or processing CSV: {e}"); traceback.print_exc(); exit()

    print("\n--- 2. Feature Engineering (Simple_Predictor_B Features) ---")
    feature_calc_start = time.time()
    df_features = calculate_selected_features(df_raw, symbol=SYMBOL_NAME)
    feature_calc_end = time.time()
    if df_features.empty: exit("Feature calculation failed. Exiting.")
    print(f"Feature calculation completed in {feature_calc_end - feature_calc_start:.2f} seconds.")
    CURRENT_FEATURE_COLS = [f for f in MODEL_FEATURE_COLS if f in df_features.columns]
    if not CURRENT_FEATURE_COLS: exit("ERROR: No modeling features found after calculation.")
    if len(CURRENT_FEATURE_COLS) < len(MODEL_FEATURE_COLS): print(f"Warning: Only {len(CURRENT_FEATURE_COLS)}/{len(MODEL_FEATURE_COLS)} modeling features generated.")
    print(f"Using {len(CURRENT_FEATURE_COLS)} features for modeling.")

    print("\n--- 3. Data Cleaning (Post-Features) ---")
    numeric_feature_cols = df_features[CURRENT_FEATURE_COLS].select_dtypes(include=np.number).columns.tolist()
    df_features[numeric_feature_cols] = df_features[numeric_feature_cols].replace([np.inf, -np.inf], np.nan)
    nan_check = df_features[numeric_feature_cols].isnull().sum()
    total_nans = nan_check.sum()
    print(f"Total NaNs found in {len(numeric_feature_cols)} numeric feature columns: {total_nans}.")

    print("\n--- 4. Modeling Target & Final Prep ---")
    TARGET_COLUMN = 'target'
    df = df_features.copy()
    df = df.sort_values('timestamp')
    if 'close' not in df.columns: exit("ERROR: 'close' column missing.")
    print(f"Creating binary target: {PREDICTION_WINDOW_HOURS}h future return >= {TARGET_THRESHOLD_PCT}%...")
    df['future_price'] = df['close'].shift(-PREDICTION_WINDOW_ROWS)
    with np.errstate(divide='ignore', invalid='ignore'):
         df['price_return_future'] = (df['future_price'] - df['close']) / df['close'].replace(0, np.nan) * 100
    df[TARGET_COLUMN] = np.where(df['price_return_future'] >= TARGET_THRESHOLD_PCT, 1, 0)
    df.loc[df['price_return_future'].isnull(), TARGET_COLUMN] = np.nan
    df = df.drop(['future_price', 'price_return_future'], axis=1)
    initial_rows = len(df)
    essential_check_cols = ['close', TARGET_COLUMN]
    df = df.dropna(subset=essential_check_cols)
    print(f"Rows after removing NaN targets/close: {len(df)} (Removed {initial_rows - len(df)})")
    rows_before_feature_nan_check = len(df)
    rows_after_feature_nan_dropna = len(df.dropna(subset=CURRENT_FEATURE_COLS))
    potential_feature_nan_loss = rows_before_feature_nan_check - rows_after_feature_nan_dropna
    if potential_feature_nan_loss > 0: print(f"Note: {potential_feature_nan_loss} rows have NaNs in features. Models/Imputer will handle.")
    if df.empty: exit("DataFrame empty after target creation/NaN drop.")
    target_counts = df[TARGET_COLUMN].value_counts(normalize=True) * 100
    print("\nTarget variable distribution:")
    print(f"  0 (< {TARGET_THRESHOLD_PCT}%): {target_counts.get(0, 0):.2f}%")
    print(f"  1 (>= {TARGET_THRESHOLD_PCT}%): {target_counts.get(1, 0):.2f}%")
    df = df.sort_values('timestamp').reset_index(drop=True)
    print(f"Final DataFrame shape for backtesting: {df.shape}")

    # --- 5. Walk-Forward Validation ---
    print("\n--- 5. Starting Walk-Forward Validation (Stacking - Logistic Regression Meta) ---")
    all_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    all_best_thresholds = []
    meta_feature_names = ['xgb_pred', 'lgbm_pred', 'svm_pred']
    # Store meta-feature importances (coefficients for LR)
    meta_feature_importances = {meta_feat: [] for meta_feat in meta_feature_names}
    iteration_count = 0
    n_rows_total = len(df)
    current_train_start_idx = 0
    total_iterations_estimate = max(0, (n_rows_total - TRAIN_WINDOW_ROWS - TEST_WINDOW_ROWS) // STEP_ROWS + 1) if STEP_ROWS > 0 else 0

    print(f"Total rows: {n_rows_total}, Train: {TRAIN_WINDOW_HOURS}h ({TRAIN_WINDOW_ROWS} rows), Eval: {TEST_WINDOW_HOURS}h ({TEST_WINDOW_ROWS} rows), Step: {STEP_HOURS}h ({STEP_ROWS} rows)")
    print(f"Estimated iterations: {total_iterations_estimate}")
    print(f"Using {len(CURRENT_FEATURE_COLS)} features.")
    print(f"Stacking Folds (K): {N_STACKING_FOLDS}")
    print(f"Meta Learner: Logistic Regression, Tuning C over: {META_LR_PARAM_GRID['C']}")
    print(f"Threshold Search Range: {THRESHOLD_SEARCH_RANGE}")
    print("-" * 30)
    start_loop_time = time.time()

    while True:
        train_end_idx = current_train_start_idx + TRAIN_WINDOW_ROWS
        test_start_idx = train_end_idx
        test_end_idx = test_start_idx + TEST_WINDOW_ROWS
        if test_end_idx > n_rows_total: print(f"\nStopping: Eval window end ({test_end_idx}) > total rows ({n_rows_total})."); break
        if current_train_start_idx >= n_rows_total: print(f"\nStopping: Train start index ({current_train_start_idx}) reached end."); break

        train_df = df.iloc[current_train_start_idx : train_end_idx].copy()
        test_df = df.iloc[test_start_idx : test_end_idx].copy()

        min_train_samples = max(50, int(0.1 * TRAIN_WINDOW_ROWS), N_STACKING_FOLDS * 5) # Need more for robust CV
        min_test_samples = 10
        if len(train_df) < min_train_samples or len(test_df) < min_test_samples:
            print(f"Skipping iter {iteration_count + 1}: Insufficient data train ({len(train_df)}/{min_train_samples}) or test ({len(test_df)}/{min_test_samples}).")
            current_train_start_idx += STEP_ROWS; continue

        X_train_full = train_df[CURRENT_FEATURE_COLS]
        y_train_full = train_df[TARGET_COLUMN]
        X_test = test_df[CURRENT_FEATURE_COLS]
        y_test = test_df[TARGET_COLUMN]

        if len(y_train_full.unique()) < 2: print(f"Skipping iter {iteration_count + 1}: Train data has only one class: {y_train_full.unique()}."); current_train_start_idx += STEP_ROWS; continue
        if len(y_test.unique()) < 2: print(f"Warning iter {iteration_count + 1}: Eval test data (size {len(test_df)}) has only one class: {y_test.unique()}. Metrics affected.")

        neg_count = y_train_full.value_counts().get(0, 0); pos_count = y_train_full.value_counts().get(1, 0)
        scale_pos_weight_val = neg_count / pos_count if pos_count > 0 else 1.0

        iter_start_time = time.time()
        print(f"\n--- Iter {iteration_count + 1}/{total_iterations_estimate} ---")
        print(f"  Train Indices: [{current_train_start_idx}:{train_end_idx-1}], Eval Indices: [{test_start_idx}:{test_end_idx-1}]")
        print(f"  Train Target Dist: {dict(y_train_full.value_counts(normalize=True))}")
        print(f"  Test Target Dist: {dict(y_test.value_counts(normalize=True))}")
        print(f"  Using scale_pos_weight for XGB/LGBM: {scale_pos_weight_val:.4f}")

        # --- Grid Search for Base Models ---
        print("  Grid searching base models...")
        best_xgb_params, xgb_score = grid_search_base_model('xgb', BASE_XGB_PARAM_GRID, X_train_full, y_train_full, scale_pos_weight_val)
        best_lgbm_params, lgbm_score = grid_search_base_model('lgbm', BASE_LGBM_PARAM_GRID, X_train_full, y_train_full, scale_pos_weight_val)
        best_svm_params, svm_score = grid_search_base_model('svm', BASE_SVM_PARAM_GRID, X_train_full, y_train_full, scale_pos_weight_val) # scale_pos not used directly for SVM pipeline
        print(f"    Best XGB Params: {best_xgb_params} (CV F1: {xgb_score:.3f})")
        print(f"    Best LGBM Params: {best_lgbm_params} (CV F1: {lgbm_score:.3f})")
        print(f"    Best SVM Params: {best_svm_params} (CV F1: {svm_score:.3f})")

        # --- Define Base Models using Best Parameters from Grid Search ---
        xgb_iter_params = XGB_BASE_PARAMS.copy(); xgb_iter_params.update(best_xgb_params or {})
        model_xgb_base = XGBClassifier(**xgb_iter_params, scale_pos_weight=scale_pos_weight_val)

        lgbm_iter_params = LGBM_BASE_PARAMS.copy(); lgbm_iter_params.update(best_lgbm_params or {})
        model_lgbm_base = LGBMClassifier(**lgbm_iter_params, scale_pos_weight=scale_pos_weight_val)

        svm_iter_grid_params = best_svm_params or {}
        pipeline_svm_base = Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
            ('svm', SVC(**SVM_BASE_PARAMS, **svm_iter_grid_params)) # Combine fixed & best grid params
        ])
        models_oof = {'xgb': model_xgb_base, 'lgbm': model_lgbm_base, 'svm': pipeline_svm_base}
        oof_arrays = {'xgb': np.full(len(train_df), np.nan), 'lgbm': np.full(len(train_df), np.nan), 'svm': np.full(len(train_df), np.nan)}

        # --- K-Fold OOF Generation ---
        print(f"  Level 0: Generating OOF predictions using {N_STACKING_FOLDS}-Fold CV...")
        skf = StratifiedKFold(n_splits=N_STACKING_FOLDS, shuffle=True, random_state=42 + iteration_count)
        for fold, (train_idx_k, val_idx_k) in enumerate(skf.split(X_train_full, y_train_full)):
            X_train_k, y_train_k = X_train_full.iloc[train_idx_k], y_train_full.iloc[train_idx_k]
            X_val_k, y_val_k = X_train_full.iloc[val_idx_k], y_train_full.iloc[val_idx_k]

            if len(np.unique(y_train_k)) < 2 or len(np.unique(y_val_k)) < 2:
                print(f"    Warning: Fold {fold+1} has single class. Assigning prior.")
                prior = y_train_full.mean(); [oof_arrays[key].__setitem__(val_idx_k, prior) for key in oof_arrays]; continue

            for name, model in models_oof.items():
                try:
                    fit_params_k = {}
                    if name == 'lgbm': fit_params_k = {'callbacks': [early_stopping(10, verbose=False), log_evaluation(0)], 'eval_metric': 'logloss', 'eval_set': [(X_val_k, y_val_k)]}
                    elif name == 'xgb': fit_params_k = {'eval_set': [(X_val_k, y_val_k)], 'early_stopping_rounds': 10, 'verbose': False}
                    model.fit(X_train_k, y_train_k, **fit_params_k)
                    oof_arrays[name][val_idx_k] = model.predict_proba(X_val_k)[:, 1]
                except Exception as e_kfold:
                    print(f"    Error in K-Fold {fold+1} for {name}: {e_kfold}"); prior = y_train_full.mean(); oof_arrays[name][val_idx_k] = prior

        X_meta_train_dict = {}
        models_failed_oof = []
        for name in models_oof:
            oof_array = oof_arrays[name]
            if np.isnan(oof_array).all(): models_failed_oof.append(name)
            mean_oof = np.nanmean(oof_array)
            if pd.isna(mean_oof): mean_oof = 0.5 # Fallback if all NaN
            oof_array = np.nan_to_num(oof_array, nan=mean_oof)
            if np.isnan(oof_arrays[name]).any(): print(f"    Imputed NaNs in OOF for {name} with mean {mean_oof:.4f}")
            X_meta_train_dict[f'{name}_pred'] = oof_array

        if models_failed_oof: print(f"  ERROR: Base models {models_failed_oof} failed OOF generation. Skipping iter."); current_train_start_idx += STEP_ROWS; continue

        X_meta_train = pd.DataFrame(X_meta_train_dict, index=X_train_full.index)
        y_meta_train = y_train_full
        print(f"  Level 0 OOF Generation Done. Meta Train Shape: {X_meta_train.shape}")

        # --- Train Base Models on Full Training Data ---
        print(f"  Level 0: Training base models on full training data ({len(train_df)} rows)...")
        models_full = {}
        all_base_trained = True
        for name, model in models_oof.items():
            try:
                params = {}
                if name == 'xgb': params['verbose'] = False
                model.fit(X_train_full, y_train_full, **params)
                models_full[name] = model
            except Exception as e_full_fit: print(f"  ERROR: Failed to train base model '{name}': {e_full_fit}"); all_base_trained = False; break
        if not all_base_trained: print("  Skipping iteration due to base model training failure."); current_train_start_idx += STEP_ROWS; continue
        print("  Level 0 Full Training Done.")

        # --- Level 1: Meta Learner (Logistic Regression) Tuning & Threshold Tuning ---
        print("  Level 1: Tuning Meta-Learner (Logistic Regression) and Probability Threshold...")
        best_meta_C = None
        best_meta_score = -np.inf
        best_meta_model_for_thresh = None
        best_threshold_iter = 0.5
        best_thresh_f1_score = -np.inf

        # --- Scale Meta Features ---
        meta_scaler = StandardScaler()
        X_meta_train_scaled = meta_scaler.fit_transform(X_meta_train)
        X_meta_train_scaled = pd.DataFrame(X_meta_train_scaled, index=X_meta_train.index, columns=X_meta_train.columns)

        meta_val_size = int(len(X_meta_train_scaled) * META_VALIDATION_PCT)
        if meta_val_size < 10 or (len(X_meta_train_scaled) - meta_val_size) < 10:
            print(f"  Warning: Meta dataset (size {len(X_meta_train_scaled)}) too small for validation split. Using default C=1.")
            best_meta_C = 1.0 # Default C
        else:
            X_meta_train_sub = X_meta_train_scaled[:-meta_val_size]
            y_meta_train_sub = y_meta_train[:-meta_val_size]
            X_meta_val = X_meta_train_scaled[-meta_val_size:]
            y_meta_val = y_meta_train[-meta_val_size:]

            if len(y_meta_val.unique()) < 2 or len(y_meta_train_sub.unique()) < 2:
                print("  Warning: Meta train/val split has single class. Using default C=1.")
                best_meta_C = 1.0
            else:
                # Meta Grid Search (Tuning C for Logistic Regression)
                print(f"    Tuning meta learner C over {META_LR_PARAM_GRID['C']}...")
                for c_val in META_LR_PARAM_GRID['C']:
                    try:
                        current_meta_params = {**META_LR_FIXED_PARAMS, 'C': c_val}
                        model_meta_cv = LogisticRegression(**current_meta_params)
                        model_meta_cv.fit(X_meta_train_sub, y_meta_train_sub) # Fit on scaled sub-train
                        y_pred_meta_val_cv = model_meta_cv.predict(X_meta_val) # Predict on scaled val
                        meta_score = f1_score(y_meta_val, y_pred_meta_val_cv, average='binary', pos_label=1, zero_division=0)

                        if meta_score >= best_meta_score: # Use >= to prefer simpler models (higher C = less regularization) if score is equal
                            best_meta_score = meta_score
                            best_meta_C = c_val
                            best_meta_model_for_thresh = model_meta_cv # Keep best model for threshold tuning
                    except Exception as e_meta_cv:
                        print(f"    Error during Meta CV with C={c_val}: {e_meta_cv}")
                        if best_meta_C is None: best_meta_C = 1.0 # Fallback if all fail

                if best_meta_C is None: best_meta_C = 1.0 # Ensure we have a C if loop failed
                print(f"    Best Meta C Found: {best_meta_C} (Validation F1: {best_meta_score:.4f})")

                # Threshold Tuning using the best LR model found
                if best_meta_model_for_thresh is not None:
                    print(f"    Tuning threshold over range {THRESHOLD_SEARCH_RANGE}...")
                    try:
                        y_meta_proba_val = best_meta_model_for_thresh.predict_proba(X_meta_val)[:, 1] # Predict proba on scaled val
                        f1_scores_thresh = {}
                        for t in THRESHOLD_SEARCH_RANGE:
                            y_pred_meta_val_t = (y_meta_proba_val >= t).astype(int)
                            current_f1 = f1_score(y_meta_val, y_pred_meta_val_t, average='binary', pos_label=1, zero_division=0)
                            f1_scores_thresh[t] = current_f1
                            if current_f1 >= best_thresh_f1_score:
                                best_thresh_f1_score = current_f1; best_threshold_iter = t
                        print(f"    Best Threshold Found: {best_threshold_iter:.2f} (Validation F1: {best_thresh_f1_score:.4f})")
                    except Exception as e_thresh: print(f"    Error during threshold tuning: {e_thresh}. Using default {best_threshold_iter:.2f}.")
                else: print(f"    Skipping threshold tuning. Using default {best_threshold_iter:.2f}.")

        # --- Level 1: Train Final Meta Learner (Logistic Regression) ---
        print("  Level 1: Training final Meta-Learner (Logistic Regression)...")
        try:
             final_meta_params = {**META_LR_FIXED_PARAMS, 'C': best_meta_C or 1.0} # Use best C or default
             meta_model_final = LogisticRegression(**final_meta_params)
             # Fit on the ENTIRE SCALED OOF training data
             meta_model_final.fit(X_meta_train_scaled, y_meta_train)
             print("  Level 1 Final Meta Training Done.")
        except Exception as e_meta_final: print(f"  ERROR: Failed to train final meta-learner: {e_meta_final}"); current_train_start_idx += STEP_ROWS; continue

        # --- Prediction Phase ---
        print("  Prediction: Generating final predictions on evaluation data...")
        try:
            # Predict with Level 0 models on Eval data
            pred_xgb_test = models_full['xgb'].predict_proba(X_test)[:, 1]
            pred_lgbm_test = models_full['lgbm'].predict_proba(X_test)[:, 1]
            pred_svm_test = models_full['svm'].predict_proba(X_test)[:, 1] # SVM pipeline handles impute/scale

            # Create Meta Features for Test Data
            X_meta_test = pd.DataFrame({'xgb_pred': pred_xgb_test, 'lgbm_pred': pred_lgbm_test, 'svm_pred': pred_svm_test}, index=X_test.index)

            # --- Scale Meta Test Features using the SAME scaler ---
            X_meta_test_scaled = meta_scaler.transform(X_meta_test)

            # Predict Probabilities with Final Meta Learner on SCALED meta test features
            y_proba_test = meta_model_final.predict_proba(X_meta_test_scaled)[:, 1]

            # Apply Tuned Threshold
            y_pred = (y_proba_test >= best_threshold_iter).astype(int)
            print("  Prediction Done.")

        except Exception as e_pred:
             print(f"  ERROR during prediction phase: {e_pred}");
             [all_metrics[key].append(np.nan) for key in all_metrics]; all_best_thresholds.append(np.nan);
             [meta_feature_importances[key].append(np.nan) for key in meta_feature_names] # Add NaN for importance too
             current_train_start_idx += STEP_ROWS; continue

        # --- Evaluation ---
        if len(np.unique(y_test)) < 2:
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0)
            recall = recall_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0)
            f1 = f1_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0)
            print(f"  Evaluation Metrics (Test Window Size: {TEST_WINDOW_HOURS}h, SINGLE CLASS {y_test.unique()[0]}): Acc={accuracy:.4f}, Prc={precision:.4f}, Rec={recall:.4f}, F1={f1:.4f}")
        else:
             accuracy = accuracy_score(y_test, y_pred)
             precision = precision_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0)
             recall = recall_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0)
             f1 = f1_score(y_test, y_pred, average='binary', pos_label=1, zero_division=0)
             print(f"  Evaluation Metrics (Test Window Size: {TEST_WINDOW_HOURS}h): Acc={accuracy:.4f}, Prc={precision:.4f}, Rec={recall:.4f}, F1={f1:.4f}")

        all_metrics['accuracy'].append(accuracy)
        all_metrics['precision'].append(precision)
        all_metrics['recall'].append(recall)
        all_metrics['f1'].append(f1)
        all_best_thresholds.append(best_threshold_iter)

        # --- Store Meta-Learner Feature Importances (Logistic Regression Coefficients) ---
        try:
            if isinstance(meta_model_final, LogisticRegression) and hasattr(meta_model_final, 'coef_'):
                # Get absolute coefficient values as importance
                abs_coeffs = np.abs(meta_model_final.coef_[0])
                if len(abs_coeffs) == len(meta_feature_names):
                    for i, name in enumerate(meta_feature_names):
                        meta_feature_importances[name].append(abs_coeffs[i])
                else:
                     print("  Warning: Meta-feature names count mismatch with LR coefficients.")
                     [meta_feature_importances[key].append(np.nan) for key in meta_feature_names]
            else:
                 print("  Meta-learner is not Logistic Regression or has no coefficients.")
                 [meta_feature_importances[key].append(np.nan) for key in meta_feature_names]
        except Exception as e_imp:
            print(f"  Warning: Could not get meta-feature importance (LR Coeffs): {e_imp}")
            [meta_feature_importances[key].append(np.nan) for key in meta_feature_names]

        iteration_count += 1
        iter_end_time = time.time()
        print(f"  Iteration {iteration_count} finished in {iter_end_time - iter_start_time:.2f} seconds.")
        print("-" * 20)
        current_train_start_idx += STEP_ROWS

    # --- End of Walk-Forward Loop ---
    end_loop_time = time.time()
    loop_duration_minutes = (end_loop_time - start_loop_time) / 60
    print("-" * 30)
    print(f"Walk-Forward Validation (Stacking - Ensemble_Method_C) finished in {end_loop_time - start_loop_time:.2f} seconds ({loop_duration_minutes:.2f} minutes).")

    # --- 6. Aggregate and Display Results ---
    print("\n--- 6. Final Results (Ensemble_Method_C) ---")
    if iteration_count > 0 and len(all_metrics['f1']) > 0:
        valid_indices = [i for i, f1 in enumerate(all_metrics['f1']) if not pd.isna(f1)]
        if valid_indices:
            valid_accuracy = [all_metrics['accuracy'][i] for i in valid_indices]
            valid_precision = [all_metrics['precision'][i] for i in valid_indices]
            valid_recall = [all_metrics['recall'][i] for i in valid_indices]
            valid_f1 = [all_metrics['f1'][i] for i in valid_indices]
            valid_thresholds = [all_best_thresholds[i] for i in valid_indices if not pd.isna(all_best_thresholds[i])]

            avg_accuracy = np.mean(valid_accuracy); avg_precision = np.mean(valid_precision)
            avg_recall = np.mean(valid_recall); avg_f1 = np.mean(valid_f1)

            print("\n--- Average Walk-Forward Validation Results ---")
            print(f"Total Iterations Run: {iteration_count}, Successful Evaluations: {len(valid_indices)}")
            print(f"Target: >= {TARGET_THRESHOLD_PCT}% increase over {PREDICTION_WINDOW_HOURS} hours")
            print(f"Train: {TRAIN_WINDOW_HOURS}h, Eval: {TEST_WINDOW_HOURS}h, Step: {STEP_HOURS}h")
            print(f"Stacking Folds: {N_STACKING_FOLDS}, Meta-Learner: Logistic Regression")
            print(f"Average Accuracy:  {avg_accuracy:.4f}")
            print(f"Average Precision: {avg_precision:.4f}")
            print(f"Average Recall:    {avg_recall:.4f}")
            print(f"Average F1-Score:  {avg_f1:.4f}")

            std_accuracy = np.std(valid_accuracy); std_precision = np.std(valid_precision)
            std_recall = np.std(valid_recall); std_f1 = np.std(valid_f1)
            print("\n--- Standard Deviation of Metrics ---")
            print(f"Std Dev Accuracy:  {std_accuracy:.4f}")
            print(f"Std Dev Precision: {std_precision:.4f}")
            print(f"Std Dev Recall:    {std_recall:.4f}")
            print(f"Std Dev F1-Score:  {std_f1:.4f}")

            if valid_thresholds:
                avg_threshold = np.mean(valid_thresholds); std_threshold = np.std(valid_thresholds)
                print(f"\nAverage Best Threshold: {avg_threshold:.3f} (StdDev: {std_threshold:.3f}) over {len(valid_thresholds)} folds")
            else: print("\nCould not determine average threshold.")

            print("\n--- Average Meta-Feature Importances (Abs Coefficient Magnitude) ---")
            avg_meta_importances = {}
            for f, imp_list in meta_feature_importances.items():
                 valid_imps = [imp for imp in imp_list if not pd.isna(imp)]
                 avg_meta_importances[f] = np.mean(valid_imps) if valid_imps else 0.0

            total_importance = sum(avg_meta_importances.values())
            if total_importance > 1e-9:
                 normalized_importances = {f: (imp / total_importance) * 100 for f, imp in avg_meta_importances.items()}
                 sorted_meta_importances = sorted(normalized_importances.items(), key=lambda item: item[1], reverse=True)
                 print("  (Normalized % Magnitude)")
                 for i, (feature, importance) in enumerate(sorted_meta_importances): print(f"  {i+1}. {feature}: {importance:.2f}%")
            else: print("  Meta-feature importance data (coefficients) could not be calculated or was always zero.")
        else: print("\nNo valid metrics recorded.")
    else: print("\nNo iterations were successfully completed or no metrics generated.")

    print("\nScript Ensemble_Method_C finished.")

--- Ensemble_Method_C (Logistic Regression Meta-Learner) ---
--- 1. Data Loading & Initial Prep ---
Loading data from: C:\Users\mason\AVP\BTCUSDrec.csv
Raw data loaded. Shape: (15177, 9)
Initial data prep done. Shape: (15177, 7)

--- 2. Feature Engineering (Simple_Predictor_B Features) ---
Starting calculation for 49 target columns (incl. base)...
  Calculating features...
    Calculating ATR, Garman-Klass, and Parkinson volatility features...
  Assembling final dataframe...
Selected feature calculation finished. Returning 15177 rows, 51 columns (43 calculated features). Took 0.04s.
Feature calculation completed in 0.04 seconds.
Using 43 features for modeling.

--- 3. Data Cleaning (Post-Features) ---
Total NaNs found in 43 numeric feature columns: 1034.

--- 4. Modeling Target & Final Prep ---
Creating binary target: 12h future return >= 0.0%...
Rows after removing NaN targets/close: 15165 (Removed 12)
Note: 183 rows have NaNs in features. Models/Imputer will handle.

Target variable 

KeyboardInterrupt: 